<h1>Задание 5. Применение дискретного ряда Фурье для решения методом сеток однородного уравнения теплопроводности</h1>
<h2 align = "center"><font color="#6f8600">Вариант 6</font></h2>
<p>Будем искать решение следующей задачи:</p>
$\frac{\partial u}{\partial t} = \frac{\partial^{2} u}{\partial x^{2}}, \ \ 0 < x < 1, \ \ 0 < t \leqslant 0.1$<br/>
$u(x, 0) = sin(3\pi x), \ \ 0 \leqslant x \leqslant 1$<br/>
$u(0, t) = 0, \ \ u(1, t) = 0,\ \ 0 < t \leqslant 0.1$<br/>
<p>Будем искать аналитическое решение, используя ряд Фурье и дискретный ряд Фурье, а также сеточное решение, используя ДРФ по схеме с весами</p>

In [1]:
# Подключение служебных пакетов
import sympy as sp
import math
import pandas as pd
import numpy as np

<h3>1) Построим аналитическое решение, используя ряд Фурье</h3>
<p>Решение может быть представлено в виде:</p>
$u(x,t) = \sqrt2 \sum\limits_{p=1}^{\infty} c_{p} e^{-\pi^{2}p^{2}t} sin(p\pi x)$, где $c_{p} = \int\limits_{0}^{1} sin(3\pi x) \sqrt2 sin(p\pi x)dx$

In [9]:
x, t = sp.symbols("x, t")
phi = sp.simplify(sp.sin(3*sp.pi*x))
uf = buildFourierSeries(phi, 30)

# Печать таблицы аналитического решения 
solution = generateTable(uf)
print(bigTable(solution))

    t/x    0       0.2       0.4       0.6       0.8    1
0  0.00  0.0  0.951057 -0.587785 -0.587785  0.951057  0.0
1  0.02  0.0  0.160942 -0.099468 -0.099468  0.160942  0.0
2  0.04  0.0  0.027235 -0.016832 -0.016832  0.027235  0.0
3  0.06  0.0  0.004609 -0.002848 -0.002848  0.004609  0.0
4  0.08  0.0  0.000780 -0.000482 -0.000482  0.000780  0.0
5  0.10  0.0  0.000132 -0.000082 -0.000082  0.000132  0.0


<h3>2) Построим аналитическое решение, используя дискретный ряд Фурье</h3>
<p>Решение может быть представлено в виде:</p>
$u(x,t) \approx \sqrt2 \sum\limits_{p=1}^{N-1} \widetilde{c}_{p} e^{-\pi^{2}p^{2}t} sin(p\pi x)$, где $\widetilde{c}_{p} = \sqrt2 h \sum\limits_{i=1}^{N-1} sin(3\pi ih) sin(p\pi i h)$

In [16]:
udsf_5 = buildDFS(phi, 5)
udsf_10 = buildDFS(phi, 10)
udsf_20 = buildDFS(phi, 20)

In [17]:
solution_5 = generateTable(udsf_5)
solution_10 = generateTable(udsf_10)
solution_20 = generateTable(udsf_20)

<p>Сравним аналитические решения, найденные при помощи ряда Фурье и при помощи дискретного ряда Фурье</p>

In [21]:
err = pd.DataFrame({ 
                   'N': [5, 10, 20],
                   '||uf-udsf^N||': [norma(solution, solution_5), norma(solution, solution_10), norma(solution, solution_20)]
                  })
print(err)

    N  ||uf-udsf^N||
0   5   5.551115e-16
1  10   1.221245e-15
2  20   1.887379e-15


<h3>3) Найдем сеточное решение, используя ДРФ по схеме с весами</h3>
<p>Решение может быть представлено в виде:</p>
$u(x,t) = \sqrt2 \sum\limits_{p=1}^{N-1} \widetilde{c}_{p} (\lambda_{p}^{\sigma})^{k} sin(p\pi x)$, где $\widetilde{c}_{p} = \sqrt2 h \sum\limits_{i=1}^{N-1} sin(3\pi ih) sin(p\pi i h)$<br/>
<p>Также </p>
$$\lambda_{p}^{\sigma} = {1- \frac{4(1-\sigma)\tau}{h^{2}}sin^{2}(\frac{p\pi h}{2}) \over 1+ \frac{4 \sigma \tau}{h^{2}}sin^{2}(\frac{p\pi h}{2})}, \ \ p = 1, \dots, N-1$$

In [102]:
h = [0.2, 0.1, 0.05, 0.05]
tau = [0.02, 0.005, 0.00125, 0.005]
test = np.zeros((4,4))
errors = np.zeros((4,4))
sigma = [0, 1, 0.5]

errors = start(solution, phi, h, tau, errors, sigma)

In [103]:
err_res = pd.DataFrame({ 
                   '(h, tau)': ["sigma = 0", "sigma = 1", "sigma = 1/2", "sigma = 1/2- h^2/(12*tau)"],
                   '(0.2, 0.02)': errors[:, 0], 
                   '(0.1, 0.005)': errors[:, 1],
                   '(0.05, 0.00125)': errors[:, 2],
                   '(0.05, 0.005)': errors[:, 3]
                  })
err_res

,"(h, tau)","(0.2, 0.02)","(0.1, 0.005)","(0.05, 0.00125)","(0.05, 0.005)"
0,sigma = 0,0.454835,0.047420,0.005963,29.611441
1,sigma = 1,0.250946,0.078171,0.037055,0.062734
2,sigma = 1/2,0.037656,0.017587,0.021650,0.000677
3,sigma = 1/2- h^2/(12*tau),0.076637,0.003778,0.000681,0.004711


In [99]:
def start(solution, phi, h, tau, errors, sigma):
    for i in range(4):
        for j in range(3):
            u = gridSolution(phi, sigma[j], h[i], tau[i])
            errors[j, i] = norma(solution, u)
        last_sigma = 0.5 - ((h[i])**2)/(12*tau[i])
        u = gridSolution(phi, last_sigma, h[i], tau[i])
        errors[3, i] = norma(solution, u)
    return errors

<h5>Функция для нахождения сеточного решения</h5>

In [90]:
def gridSolution(phi, sigma, h, tau):
    x, k = sp.symbols("x k")
    series = 0 
    N = int(1/h)
    for i in range(1, N):
        c_p = count_coeff(phi, N, i, h)
        lambda_k = countLambda(i, sigma, h, tau)
        psi_p = sp.simplify(sp.sqrt(2)*sp.sin(i*x*sp.pi))
        series += c_p*((lambda_k)**k)*psi_p
    M = int(0.1/tau)
    res = constructGrid(series, N, M, h)
    return res

<h5>Функция для создания сетки решения, полученного сеточным методом</h5>

In [60]:
def constructGrid(series, N, M, h):
    x, k = sp.symbols("x k")
    res = np.zeros((6,6))
    for i in range(6):
        k_i = i*int(M/5)
        for j in range(1, 5):
            x_j = j*0.2
            res[i,j] = series.subs([(x, x_j), (k, k_i)])
        res[i, 0] = 0
        res[i, 5] = 0
    return res

<h5>Функция для нахождения собственных чисел $\lambda_{p}^{\sigma}$</h5>

In [45]:
def countLambda(p, sigma, h, tau):
    countSin = (math.sin((p*math.pi*h)/2))**2
    res = (1-((4*(1-sigma)*tau)/(h**2))*countSin)/(1+((4*sigma*tau)/(h**2))*countSin)
    return res

<h5>Функция для построения дискретного ряда Фурье</h5>

In [11]:
def buildDFS(phi, N):
    x, t = sp.symbols("x, t")
    series = 0
    h = 1/N
    for i in range(1, N):
        c_p = count_coeff(phi, N, i, h)
        exp_p = sp.simplify(sp.exp(-((sp.pi)**2)*(i**2)*t))
        psi_p = sp.simplify(sp.sqrt(2)*sp.sin(i*x*sp.pi))
        series += c_p*exp_p*psi_p
    return series

<h5>Функция для нахождения коэффициентов $\widetilde{c}_{p} = \sqrt2 h \sum\limits_{i=1}^{N-1} sin(3\pi ih) sin(p\pi i h)$ дискретного ряда Фурье</h5>

In [10]:
def count_coeff(phi, N, p, h):
    x = sp.Symbol("x")
    series = 0
    psi = sp.simplify(sp.sin(x*p*sp.pi))
    for i in range(1, N):
        series += phi.subs(x, i*h)*psi.subs(x, i*h)
    series = sp.simplify(series*sp.sqrt(2)*h)
    return series

<h5>Функция для построения ряда Фурье</h5>

In [3]:
def buildFourierSeries(phi, limit):
    x, t = sp.symbols("x, t")
    series = 0
    for i in range(1, limit+1):
        psi = sp.simplify(sp.sqrt(2)*sp.sin(i*x*sp.pi))
        c_p = scalMultiply(phi, psi)
        exp_p = sp.simplify(sp.exp(-((sp.pi)**2)*(i**2)*t))
        series += c_p*exp_p*psi
    return series

<h5>Функция для нахождения скалярного произведения в $L_{2}$</h5>

In [2]:
def scalMultiply(a, b):
    x = sp.Symbol("x")
    f = sp.integrate(sp.nsimplify(a*b), (x, 0, 1))
    return f

<h5>Функция для генерации таблицы значений решения на “крупной” сетке</h5>

In [4]:
def generateTable(u):
    x, t = sp.symbols("x t")
    res = np.zeros((6, 6))
    for j in range(6):
        t_j = 0.02*j 
        for i in range(6):
            x_i = 0.2*i
            res[j, i] = u.subs([(x, x_i), (t, t_j)])
    return res

<h5>Функция для печати таблицы решения на “крупной” сетке</h5>

In [5]:
def bigTable(u):
    
    df = pd.DataFrame({ 
                   't/x': [0, 0.02, 0.04, 0.06, 0.08, 0.1],
                   '0': u[:, 0], 
                   '0.2': u[:, 1],
                   '0.4': u[:, 2],
                   '0.6': u[:, 3],
                   '0.8': u[:, 4],
                   '1': u[:, 5] 
                  })
    
    return df

<h5>Функция для поиска нормы</h5>

In [19]:
def norma(u_Ex, u_Sol):
    u = u_Ex - u_Sol;
    maxV = -np.inf
    for i in range(6):
        for j in range(6):
            if abs(u[i,j]) > maxV:
                maxV = abs(u[i,j])
    return maxV